# Setup

In [ ]:
import geopandas as gpd
import pandas as pd
import leafmap as leafmap
from shapely.ops import unary_union
from shapely.geometry import Point, mapping, box, shape
import shapely
from typing import List
import os
from tqdm import tqdm
tqdm.pandas()

import sys
sys.path.append("..")

os.getcwd()
os.chdir("..")
root = os.path.dirname(os.getcwd())
# root = root + "/workspaces/mine-segmentation" # uncomment when running in Lightning Studios
root

In [ ]:
from src.data.get_satellite_images import ReadSTAC

In [ ]:
# INPUT DATASET
processed_dataset = root + "/data/processed/mining_tiles_with_masks_and_bounding_boxes.gpkg"
# OUTPUT DATASET
test_dataset_annotations = root + "/data/raw/mining_tiles_test_annotations.gpkg"

In [ ]:
# load the filtered dataset
tiles = gpd.read_file(processed_dataset, layer="tiles")
polygons = gpd.read_file(processed_dataset, layer="preferred_polygons")

# only the test data
tiles = tiles[tiles["split"] == "test"]
polygons = polygons[polygons["tile_id"].isin(tiles["tile_id"])]

In [ ]:
if not os.path.exists(test_dataset_annotations):
    tiles.to_file(test_dataset_annotations, layer="tiles", driver="GPKG")
    polygons.to_file(test_dataset_annotations, layer="polygons_original", driver="GPKG")

    # create a new layer for the annotated polygons, which is the same as polygons, but the geometry is empty
    polygons_annotated = polygons.copy()
    polygons_annotated["geometry"] = None
    polygons_annotated.to_file(test_dataset_annotations, layer="polygons_annotated", driver="GPKG")
    print("File created")
else:
    
    print("File already exists")

# Functions

In [ ]:
# Function to remove holes from a polygon
from shapely.geometry import MultiPolygon, Polygon
def remove_holes(polygon):
    if isinstance(polygon, Polygon):
        return Polygon(polygon.exterior)
    return polygon

In [ ]:
def get_tile_index():
    # load the dataset
    polygons_annotated = gpd.read_file(test_dataset_annotations, layer="polygons_annotated")

    # get the first tile for which the geometry is None
    tile = polygons_annotated[polygons_annotated["geometry"].isna()].iloc[0]

    # get index of the tile
    index = polygons_annotated[polygons_annotated["geometry"].isna()].index[0]

    print(f"Tile index: {index}, Tile ID: {tile['tile_id']}")

    return index

In [ ]:
def plot_tile_on_map(index: int, tiles: gpd.GeoDataFrame, polygons: gpd.GeoDataFrame, add_satellite=True):
    # plot that tile on a map
    m = leafmap.Map(
        center=[tiles.geometry.centroid.y.iloc[index], tiles.geometry.centroid.x.iloc[index]], 
        zoom=25,
        height="900px"
    )

    if add_satellite:
        m.add_basemap("SATELLITE")

    # visualize the tile boundaries
    style_tile = {
        "stroke": True,
        "color": "orange",
        "weight": 2,
        "opacity": 1,
        "fill": False,
    }

    # add the tile to the map
    m.add_gdf(tiles.iloc[index:index+1,:], layer_name="tiles", style=style_tile)

    # add the polygons to the map
    multipolygon = polygons.iloc[index:index+1,:]
    polygon_list = list(multipolygon.geometry.values[0].geoms)
    m.edit_vector(gpd.GeoSeries(polygon_list).to_json())

    # add the S2 image to the map
    s2_name = tiles.iloc[index:index+1,:].s2_tile_id.values[0]
    api_url="https://planetarycomputer.microsoft.com/api/stac/v1"
    stac_reader = ReadSTAC(api_url)
    bounds = tiles.iloc[index:index+1,:].geometry.bounds.values[0]
    item = stac_reader.get_item_by_name(s2_name, bbox=bounds)

    m.add_cog_layer(item.assets["visual"].href, name="Sentinel-2")

    return m, polygon_list

In [ ]:
def save_features(m, polygon_list, rm_holes=False):

    # convert to geopandas dataframe
    draw_features = gpd.GeoDataFrame.from_features(m.draw_features)

    # optionally remove any holes from the polygons
    if rm_holes:
        for i, feature in draw_features.iterrows():
            geom = feature.geometry
            if isinstance(geom, MultiPolygon):
                new_polygons = [remove_holes(p) for p in geom]
                draw_features.at[i, 'geometry'] = MultiPolygon(new_polygons)
            elif isinstance(geom, Polygon):
                draw_features.at[i, 'geometry'] = remove_holes(geom)

    print(f"Number of features before review: {len(polygon_list)}")
    print(f"Number of features after review: {len(draw_features)}")

    # convert to multipolyon
    output = draw_features["geometry"].unary_union
    return output

In [ ]:
def rm_features(m):

    # convert to geopandas dataframe
    draw_features = gpd.GeoDataFrame.from_features(m.draw_features)

    original_geom = draw_features.geometry[0]
    if len(draw_features) > 2:
        geom_to_remove = draw_features.geometry[1:].unary_union
    else:
        geom_to_remove = draw_features.geometry[1]

    # only get the area from the first polygon, that does not intersect with the second polygon
    output = original_geom.difference(geom_to_remove)
    output = output
    return output

In [ ]:
def plot_edits_on_map(m, multipolygon, tiles, index):
    # plot that tile on a map
    m = leafmap.Map(
        center=[tiles.geometry.centroid.y.iloc[index], tiles.geometry.centroid.x.iloc[index]], 
        zoom=20,
        height="600px"
    )

    # visualize the tile boundaries
    style_tile = {
        "stroke": True,
        "color": "orange",
        "weight": 2,
        "opacity": 1,
        "fill": False,
    }

    style_polygon = {
        "stroke": True,
        "color": "red",
        "weight": 2,
        "opacity": 1,
        "fill": True,
        "fillColor": "red",
        "fillOpacity": 0.1,
    }

    # add the tile to the map
    m.add_gdf(tiles.iloc[index:index+1,:], layer_name="tiles", style=style_tile)

    # add the polygons to the map
    # m.add_gdf(
    #     gdf = gpd.GeoDataFrame([{'geometry': multipolygon}], crs="EPSG:4326"),
    #     layer_name="draw_features", 
    #     style=style_polygon
    #     )
    

    m.edit_vector(gpd.GeoSeries(multipolygon).to_json(), layer_name="draw_features", style=style_polygon)

    # add the S2 image to the map
    s2_name = tiles.iloc[index:index+1,:].s2_tile_id.values[0]
    api_url="https://planetarycomputer.microsoft.com/api/stac/v1"
    stac_reader = ReadSTAC(api_url)
    bounds = tiles.iloc[index:index+1,:].geometry.bounds.values[0]
    item = stac_reader.get_item_by_name(s2_name, bbox=bounds)

    m.add_cog_layer(item.assets["visual"].href, name="Sentinel-2")

    return m

# Validate the test tiles
The aim is to visualize each tile and its mask in the notebook. 
Then, a new polygon is drawn according to my judgement, where the mine actually is. 
This new corrected polygon must then be saved. 
The framework used for this is leafmap. 

# USER INTERFACE

### REFRESH TILE

In [ ]:
index = get_tile_index()

### REVIEW
add features

In [ ]:
m, polygon_list = plot_tile_on_map(index, tiles, polygons, add_satellite=True)
m

In [ ]:
rm_holes = False
output = save_features(m, polygon_list, rm_holes=rm_holes)
output

### REVIEW
remove features

In [ ]:
m = plot_edits_on_map(m, output, tiles, index)
m

In [ ]:
output = rm_features(m)
output

In [ ]:
m = plot_edits_on_map(m, output, tiles, index)
m

# SAVE FEATURES

In [ ]:
polygons_annotated = gpd.read_file(test_dataset_annotations, layer="polygons_annotated")
polygons_annotated.loc[index, "geometry"] = output
polygons_annotated.to_file(test_dataset_annotations, layer="polygons_annotated", driver="GPKG")
polygons_annotated

# CAUTION: Remove the just annotated feature

In [ ]:
# polygons_annotated = gpd.read_file(test_dataset_annotations, layer="polygons_annotated")
# polygons_annotated.loc[index, "geometry"] = None
# polygons_annotated.to_file(test_dataset_annotations, layer="polygons_annotated", driver="GPKG")
# polygons_annotated